In [1]:
import tensorflow
import numpy
from numpy import *
numpy.random.seed(123)
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import linear_model

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras

from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


# Preprocess

In [2]:
df = pd.read_csv("Data.csv", low_memory=False)
df.shape

(39508, 389)

In [3]:
df.head(10)

,ActIntAmusementParkVisit,ActIntZooVisit,ActIntWineLovers,ActIntDoityourselfers,ActIntHomeImproveSpenders,ActIntHuntingEnthusiasts,BuyerLuxuryStoreShop,LifestyleMedicalPolicy,ActIntDigitalMagNewspaper,ActIntAttendsEducationProg,...,MortgageLoanType,ESTCURRMTHLYMORTGPAYMTRNG,MosaicGlobalZip4,PresenceOfCreditCard,InterestInCrafts,InterestInGourmetCooking,ComputersPeripherals,HiTechOwner,InternetOnlineSubscriber,Flag
0,26.0,68.0,65.0,37.0,53.0,41.0,40.0,69.0,78.0,67.0,...,NaN,0U,E,U,Y,Y,Y,Y,Y,0
1,47.0,46.0,2.0,31.0,9.0,68.0,5.0,14.0,48.0,10.0,...,C,1M,B,Y,U,U,Y,Y,Y,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,15.0,47.0,66.0,37.0,37.0,22.0,53.0,51.0,62.0,20.0,...,NaN,3N,F,U,U,Y,Y,Y,Y,0
6,20.0,31.0,63.0,11.0,46.0,42.0,42.0,72.0,49.0,42.0,...,NaN,1E,F,U,U,U,Y,Y,Y,0
7,47.0,43.0,99.0,63.0,80.0,69.0,38.0,93.0,40.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Since we have imbalance data, we drop rows that have number of Nan > 350 in order to balance the data

In [5]:
# df.dropna(how='all')

## drop rows that have number of Nan > 350 in order to balance the data
for i in range(len(df.index)) :
    #print("Nan in row ", i , " : " ,  df.iloc[i].isnull().sum())
    if df.loc[i].isnull().sum() > 350:
        df.drop(i , inplace=True)

In [5]:
df.shape

(38694, 389)

In [6]:
## get rid of the column that has maximum number of NaN
#df.count().idxmin()
maxNan = df.isnull().sum().idxmax()
df = df.drop(columns=maxNan)

In [7]:
#features = data.iloc[:,:-1]
#label = data.iloc[:,-1]
## drop only if ALL columns are NaN, and convert Nan to 0
#features = features.dropna(how='all').fillna(0)    
#df = pd.concat([features, label], axis=1)
#df.iloc[:,:-1].shape[0]

In [8]:
## Porportion of majority and minority

f_1 = round((sum(df['Flag'])/df.iloc[:,:-1].shape[0])*100,2)
f_0 = 100 - f_1
print("Flag0 porportion is" ,f_0,"%")

print("Flag1 porportion is",f_1,"%")

Flag0 porportion is 89.41 %
Flag1 porportion is 10.59 %


In [9]:
# Resample because we have imbalance data
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.Flag==0]
df_minority = df[df.Flag==1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=5000,   # to match minority class
                                 random_state=123) # reproducible results
                                 

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority]).reset_index(drop=True)

# Display new class counts
df_downsampled.Flag.value_counts()

0    5000
1    4096
Name: Flag, dtype: int64

In [10]:
## Porportion of majority and minority after balance the data

f_1 = round((sum(df_downsampled['Flag'])/df_downsampled.iloc[:,:-1].shape[0])*100,2)
f_0 = 100 - f_1
print("Flag0 porportion is" ,f_0,"%")

print("Flag1 porportion is",f_1,"%")

Flag0 porportion is 54.97 %
Flag1 porportion is 45.03 %


#### Seperate the downsample target

In [11]:
X = df_downsampled.drop(columns=['Flag'])
y = df_downsampled['Flag'].values
y.shape

(9096,)

#### Seperate the continuous variables

In [12]:
continuous = X.select_dtypes(include=["float64"]).apply(lambda x: x.fillna(x.mean()),axis=0)
continuous.shape

(9096, 319)

In [13]:
# Normalize the continuous variables
f = continuous.iloc[:,:-1]
dataf=((f-f.min())/(f.max()-f.min()))
dataf.shape

(9096, 318)

In [14]:
# Use Lasso to select feature set
clf = linear_model.Lasso(alpha=0.01, max_iter=20000)

In [15]:
clf.fit(dataf, y)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=20000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [16]:
# Note intercept
clf.intercept_

0.4448852637315671

In [17]:
pd.options.display.max_rows = 284
coef_df = pd.DataFrame({'Variable':list(dataf), 'Coef':clf.coef_})
coef_df

,Variable,Coef
0,ActIntAmusementParkVisit,-0.0
1,ActIntZooVisit,-0.0
2,ActIntWineLovers,-0.0
3,ActIntDoityourselfers,-0.0
4,ActIntHomeImproveSpenders,-0.0
...,...,...
313,ForeignvsDomestic,0.0
314,MorevsLessExpensive,0.0
315,LuxuryvsMidPriced,0.0
316,NewvsUsed,0.0


In [18]:
# Drop the features where the coefficient = 0
trimContin = coef_df[coef_df.Coef != 0].Variable
trimContin

44     StyleHighFreqDomesticVac
46     StyleFrequentFlyerPrgMbr
143             BuiltHUMedHUAge
199          UrbanPopUrbanAreas
201             EducISPSADecile
203     IncFamilyIncStateDecile
278            TTTrendsettersHH
Name: Variable, dtype: object

#### Categorical Variables

In [19]:
categorical = df_downsampled.select_dtypes(include=["object"])
#categorical = df.select_dtypes(exclude=["float64","int64"])
categorical.head()


,DiscoverPremium,OtherCardPremium,OtherCardRegular,VisaRegular,CDsMoneyMarketCurrently,Vetern,ComputerOrElectronics,HomeAndGarden,EstimatedIncomeRangeV6,LevLatLong,...,MortgageRateType,MortgageLoanType,ESTCURRMTHLYMORTGPAYMTRNG,MosaicGlobalZip4,PresenceOfCreditCard,InterestInCrafts,InterestInGourmetCooking,ComputersPeripherals,HiTechOwner,InternetOnlineSubscriber
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,R,...,NaN,NaN,3U,F,U,U,U,U,U,U
1,U,U,U,U,U,U,U,U,D,R,...,NaN,NaN,0U,G,U,Y,Y,Y,U,U
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,U,U,Y,Y,U,U,U,U,F,R,...,F,C,1J,C,Y,U,U,Y,Y,Y
4,U,Y,Y,U,U,Y,U,U,B,R,...,NaN,NaN,3U,G,Y,U,Y,Y,Y,Y


In [20]:
## fill NaN value with mode
for i in range(categorical.shape[1]):
    m = categorical.iloc[:,i].mode()
    categorical.iloc[:,i] = categorical.iloc[:,i].fillna(m[0])


/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [21]:
categorical.head().T

,0,1,2,3,4
DiscoverPremium,U,U,U,U,U
OtherCardPremium,U,U,U,U,Y
OtherCardRegular,U,U,U,Y,Y
VisaRegular,U,U,U,Y,U
CDsMoneyMarketCurrently,U,U,U,U,U
Vetern,U,U,U,U,Y
ComputerOrElectronics,U,U,U,U,U
HomeAndGarden,U,U,U,U,U
EstimatedIncomeRangeV6,D,D,E,F,B
LevLatLong,R,R,T,R,R


In [22]:
y = df_downsampled["Flag"]
y.shape

(9096,)

In [23]:
df_downsampled.shape

(9096, 388)

In [24]:
categorical = categorical.astype('category')
categorical.values.astype(str)

array([['U', 'U', 'U', ..., 'U', 'U', 'U'],
       ['U', 'U', 'U', ..., 'Y', 'U', 'U'],
       ['U', 'U', 'U', ..., 'Y', 'Y', 'Y'],
       ...,
       ['U', 'U', 'Y', ..., 'Y', 'Y', 'Y'],
       ['U', 'U', 'U', ..., 'Y', 'Y', 'Y'],
       ['U', 'U', 'U', ..., 'Y', 'Y', 'Y']], dtype='<U2')

### We use the Lasso regularisation to remove non-important ordinal features from the dataset.

In [25]:
#dummy = pd.get_dummies(data=categorical, drop_first=True)
dummy = pd.get_dummies(data=categorical)
dummy.head(10)

,DiscoverPremium_U,DiscoverPremium_Y,OtherCardPremium_U,OtherCardPremium_Y,OtherCardRegular_U,OtherCardRegular_Y,VisaRegular_U,VisaRegular_Y,CDsMoneyMarketCurrently_U,CDsMoneyMarketCurrently_Y,...,InterestInCrafts_U,InterestInCrafts_Y,InterestInGourmetCooking_U,InterestInGourmetCooking_Y,ComputersPeripherals_U,ComputersPeripherals_Y,HiTechOwner_U,HiTechOwner_Y,InternetOnlineSubscriber_U,InternetOnlineSubscriber_Y
0,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
1,1,0,1,0,1,0,1,0,1,0,...,0,1,0,1,0,1,1,0,1,0
2,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,0,1,0,1,0,1
3,1,0,1,0,0,1,0,1,1,0,...,1,0,1,0,0,1,0,1,0,1
4,1,0,0,1,0,1,1,0,1,0,...,1,0,0,1,0,1,0,1,0,1
5,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
6,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,0,1,0,1,0,1
7,1,0,1,0,0,1,1,0,1,0,...,1,0,1,0,0,1,0,1,1,0
8,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,0,1,0,1,0,1
9,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0


In [26]:
dataFrame = pd.concat([continuous,dummy,y],axis=1)

In [27]:
## normalization

f = continuous.iloc[:,:-1]
dataf=((f-f.min())/(f.max()-f.min()))
#dataf.corr(method ='pearson')
dataf.head()

,ActIntAmusementParkVisit,ActIntZooVisit,ActIntWineLovers,ActIntDoityourselfers,ActIntHomeImproveSpenders,ActIntHuntingEnthusiasts,BuyerLuxuryStoreShop,LifestyleMedicalPolicy,ActIntDigitalMagNewspaper,ActIntAttendsEducationProg,...,MortgageTerminMonths,ESTCURLOANTOVALUERATIORG,AmericanOrigin,EuropeanOrigin,AsianOrigin,ForeignvsDomestic,MorevsLessExpensive,LuxuryvsMidPriced,NewvsUsed,MidRangeCarvsTrucks
0,0.540816,0.540816,1.000000,0.653061,0.928571,0.683673,0.397959,0.979592,0.397959,0.204082,...,0.000000,0.96000,0.518581,0.076562,0.477567,0.564098,0.466301,0.334387,0.296189,0.502617
1,0.571429,0.632653,0.377551,0.367347,0.622449,0.418367,0.612245,0.551020,0.734694,0.673469,...,0.000000,0.00000,0.532139,0.142941,0.417920,0.523625,0.525796,0.725866,0.317345,0.406786
2,0.402892,0.423871,0.644661,0.500556,0.590301,0.491132,0.409360,0.632887,0.432168,0.356612,...,0.216414,0.49298,0.454820,0.133194,0.508910,0.617464,0.492357,0.403695,0.339287,0.517100
3,0.000000,0.030612,0.418367,0.479592,0.489796,0.744898,0.244898,0.377551,0.397959,0.030612,...,0.967742,0.33120,0.375862,0.122381,0.602270,0.714950,0.688255,0.484188,0.401101,0.532756
4,0.551020,0.816327,0.969388,0.489796,0.581633,0.418367,0.540816,0.989796,0.448980,0.897959,...,0.000000,0.96000,0.615180,0.126297,0.338744,0.493029,0.380900,0.266644,0.156064,0.419599


In [28]:
dataf.shape

(9096, 318)

In [29]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=250, n_jobs=-1, random_state=123)
#forest = ExtraTreesClassifier(n_estimators=250,random_state=0)
forest.fit(dataf, y)
importances = forest.feature_importances_
dfscores = pd.DataFrame(importances)
dfcolumns = pd.DataFrame(dummy.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
feature_selected = featureScores.nlargest(200,'Score')
print(feature_selected)  #print 200 best features


                           Specs     Score
273  Person6OccupationGroupV2_61  0.011419
159               I2PersonType_I  0.007897
272   Person6OccupationGroupV2_6  0.007469
238               I8PersonType_2  0.007155
271  Person6OccupationGroupV2_52  0.006821
274  Person6OccupationGroupV2_62  0.006715
191               I4GenderCode_0  0.006694
269   Person6OccupationGroupV2_5  0.006370
185               I3GenderCode_U  0.006324
204               I5PersonType_0  0.006302
158               I2PersonType_E  0.006243
267   Person6OccupationGroupV2_4  0.006001
314     PoliticalAffiliation2_5D  0.005826
194               I4GenderCode_3  0.005698
100   HouseholdCompositionCode_I  0.005624
127          CombinedHomeowner_8  0.005375
111               I1PersonType_I  0.005321
161               I2PersonType_O  0.005314
192               I4GenderCode_1  0.005230
209               I5GenderCode_F  0.005219
149    MultiCoDirectMailRespHH_M  0.005162
154               I2PersonType_2  0.005094
268  Person

In [30]:
feature_selected.index

Int64Index([273, 159, 272, 238, 271, 274, 191, 269, 185, 204,
            ...
             22,  61, 292,  48,  54,  20, 237,  32,   8,  99],
           dtype='int64', length=200)

In [31]:
selected_col = feature_selected.index

In [32]:
df_go = dataf.iloc[:,selected_col]
df_go.shape

(9096, 200)

In [33]:
df_go = pd.concat([df_go,dataf[trimContin],y],axis=1)

In [34]:
import tensorflow
import numpy
numpy.random.seed(123)
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

y = dataFrame['Flag']
# Split the data into 40% test and 60% training
X_train, X_test, y_train, y_test = train_test_split(df_go.values, y, test_size=0.3, random_state=123)

print("Dimension of the training input is: ", X_train.shape)
print("Dimension of the test input is: ", X_test.shape)

print("Dimension of the training target is: ", y_train.shape)
print("Dimension of the test target is: ", y_test.shape)

Dimension of the training input is:  (6367, 208)
Dimension of the test input is:  (2729, 208)
Dimension of the training target is:  (6367,)
Dimension of the test target is:  (2729,)


In [35]:
## one-hot encoding 
# ( everytime when we're dealing with categorical features or classifcation problem where our output is label)
## dummy variable
y_train = keras.utils.to_categorical(y_train,2)
y_test = keras.utils.to_categorical(y_test,2)
print('Dimension of the training target is :', y_train.shape)
print('Dimension of the test target is :', y_test.shape)

Dimension of the training target is : (6367, 2)
Dimension of the test target is : (2729, 2)


In [36]:
# Defining the Model

model = Sequential()
model.add(Dense(100,activation = 'relu', input_dim = 208))
model.add(Dropout(0.2))
model.add(Dense(100, activation = 'relu')) 
model.add(Dropout(0.2)) # 'dropout' one way of regularize nueron network
model.add(Dense(2, activation = 'softmax')) ## 'softmax' We are defining the label for categorical problem

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               20900     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 31,202
Trainable params: 31,202
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop',metrics=['accuracy'])

In [39]:
model.fit(X_train,y_train, batch_size =700, epochs = 5, validation_split = 1/6, verbose=1)

Train on 5305 samples, validate on 1062 samples
Epoch 1/5
5305/5305 [==============================] - 1s 208us/step - loss: 0.6339 - accuracy: 0.6190 - val_loss: 0.5380 - val_accuracy: 0.5942
Epoch 2/5
5305/5305 [==============================] - 0s 31us/step - loss: 0.5294 - accuracy: 0.7118 - val_loss: 0.4280 - val_accuracy: 0.8136
Epoch 3/5
5305/5305 [==============================] - 0s 30us/step - loss: 0.4450 - accuracy: 0.8057 - val_loss: 0.3766 - val_accuracy: 0.8211
Epoch 4/5
5305/5305 [==============================] - 0s 29us/step - loss: 0.3942 - accuracy: 0.8394 - val_loss: 0.2897 - val_accuracy: 0.9426
Epoch 5/5
5305/5305 [==============================] - 0s 29us/step - loss: 0.3337 - accuracy: 0.8971 - val_loss: 0.3640 - val_accuracy: 0.7637


In [40]:
score = model.evaluate(X_test, y_test)
# model.predict will give us the y-hat (prediction)
print("\nModel", model.metrics_names[1], 'on test set is', score[1])

2729/2729 [==============================] - 0s 49us/step

Model accuracy on test set is 0.7486258745193481


In [41]:
## find prediction
yhat = numpy.round(model.predict(X_test),0)
yhat

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [42]:
yhat.shape

(2729, 2)

In [43]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

results = confusion_matrix(y_test[:,0], yhat[:,0]) 

In [44]:
print('Confusion Matrix :')
print(results) 
print("")
print( 'Accuracy Score :',accuracy_score(y_test[:,0], yhat[:,0]))
print("")
print('Report : ')
print(classification_report(y_test[:,0], yhat[:,0])) 

Confusion Matrix :
[[ 560  686]
 [   0 1483]]

Accuracy Score : 0.7486258702821547

Report : 
              precision    recall  f1-score   support

         0.0       1.00      0.45      0.62      1246
         1.0       0.68      1.00      0.81      1483

    accuracy                           0.75      2729
   macro avg       0.84      0.72      0.72      2729
weighted avg       0.83      0.75      0.72      2729



In [45]:
# Import Joblib Module from Scikit Learn

from sklearn.externals import joblib

# Save the model as a pickle in a file
joblib_file = "FinalModel.pkl"  
joblib.dump(model, joblib_file)

/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['FinalModel.pkl']

In [46]:
# Load the model from the file 
knn_from_joblib = joblib.load('FinalModel.pkl')  
  
# Use the loaded model to make predictions 
knn_from_joblib.predict(X_test) 

array([[0.37067518, 0.6293248 ],
       [0.49767113, 0.5023289 ],
       [0.96656793, 0.03343213],
       ...,
       [0.4506498 , 0.54935014],
       [0.9571413 , 0.04285876],
       [0.5568049 , 0.44319513]], dtype=float32)

# Report

correctly classify for the majority class

Summary

- get rid of the column that has maximum number of NaN
- drop rows that have number of Nan > 350 
- We downsize our sample to balance the data.
- For numerical variables, we convert Nan value to zero.
- For categorical variables, we convert Nan value to column mode.
- Classification Rate/Accuracy is 